# Zillow Logerror Prediction

## Key Takeaways



## Contents <a name='contents'></a>

*Hyperlinks will only work on locally stored copies of this Jupyter Notebook*

1. <a href='#intro'>Introduction</a>
2. <a href='#wrangle'>Wrangle</a>
    1. <a href='#acquire'>Acquire the Data</a>
    2. <a href='#prepare'>Prepare the Data</a>
3. <a href='#explore'>Explore</a>
    1. <a href='#target'>Looking at the Target Variable</a>

In [1]:
#imports

import wrangle_zillow

## Introduction <a name='intro'></a>

Zillow data was pulled from the database, and analyzed to determine which features can best determine the logerror of the prediction model. This project is divided into three parts: wrangle, explore, and model. Wrangle explains how the data is acquired and prepared for analysis and processing. Explore looks at the data and applies visualization and hypothesis testing to discover drivers of the logerror.  Clusdtering is used at this stage in order to find groups of rows that are similar. Finally, model builds a model to predict logerror from the data. Each section and select subsections include a short list of key takeaways; these are followed by a discussion detailing the analysis choices made that are relevant to that (sub)section.

### Goals
- Use clustering algorithms to help determine predictors of logerror to help improve the performance of a property value model
- Using drivers of logerror to improve a model of property values
- Improve understanding of logerror to better inform the use of models for property prediction

### Project Plan
- Explore at least four variables of property prices
- Visualize variables
- Hypothesis test at least two variables
- Write this final report
- Python scripts that allow for the project to be reproducible

<a href='#contents'>Back to contents</a>

## Wrangle <a name='wrangle'></a>

### Key Wrangle Takeaways
- $0.66$ of the data is retained, the majority of the dropped data ($0.32$) is from focusing on single unit properties
- Final dataframe has 28 columns and 51606 rows
- There is no missing data in the target `logerror` column

### Acquire the data <a name='acquire'></a>

#### Key Acqure Takeaways
- Data is acquired via `wrangle_zillow.get_zillow_data()`
- Original dataframe is 68 columns and 77414 rows
- There is significant missing data, but none in the `logerror` column

#### Discussion

Data is acquired via the `wrangle_zillow.get_zillow_data()` function. This function will query the SQL database, unless there is a saved .csv file present in the current directory. The name of the file is set by the `wrangle_zillow.FILENAME` constant. This function has the following parameters:

- `query_db = False` (bool) : forces a query to the SQL database even if the .csv file is present.  This will overwrite the existing .csv file.

In order to determine the extent of missing data, `wrangle_zillow.return_col_percent_null()` is used. The function has the followng parameters:

- `df` (DataFrame) : a dataframe containing the Zillow data

This function will return a dataframe with the following columns:

- `column_name` : The name of the column of the relevant column of df
- `percent_null` : The percent of rows in the column_name column of df that are null values
- `count_null` : The total number of null values in the column_name column of df

<a href='#contents'>Back to contents</a>

The first step is to aquire the data via `wrangle_zillow.get_zillow_data()`:

In [2]:
#use get_zillow_data() to acquire the data
df = wrangle_zillow.get_zillow_data()

Returning saved csv file.


/Users/stephenfitzsimon/codeup-data-science/zillow_clustering_project/wrangle_zillow.py:37: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(FILENAME).drop(columns = ['Unnamed: 0'])


Take a look at what the data looks like:

In [3]:
#look at three rows of the dataframe
df.sample(3)

,logerror,transdate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,typeconstructiondesc,storydesc,propertylandusedesc,heatingorsystemdesc,airconditioningdesc,architecturalstyledesc,buildingclassdesc
54327,0.096432,2017-07-06,1053926,13073408,1.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,6.037403e+13,NaN,NaN,Condominium,Central,Central,NaN,NaN
54815,-0.026412,2017-07-07,2076472,11288338,1.0,NaN,NaN,2.0,3.0,NaN,...,NaN,NaN,6.037901e+13,NaN,NaN,Single Family Residential,Central,Central,NaN,NaN
76491,-0.009062,2017-09-14,315119,10900980,NaN,NaN,NaN,2.0,3.0,NaN,...,NaN,NaN,6.037143e+13,NaN,NaN,Single Family Residential,Central,NaN,NaN,NaN


Determine the shape of the data, and store it in a variable for later use.

In [4]:
#get the shape attribute of the dataframe and store it in a variable.
original_shape = df.shape
original_shape

(77414, 68)

Use `wrangle_zillow.return_col_percent_null()` to determine the number of null values in each column.

In [5]:
#use return_col_percent_null to get a dataframe with percent and count of null per column
#sort by the percent of the column that is null
null_prop_df = wrangle_zillow.return_col_percent_null(df).sort_values('percent_null')
null_prop_df

,column_name,percent_null,count_null
0,logerror,0.0000,0
1,transdate,0.0000,0
2,id,0.0000,0
3,parcelid,0.0000,0
20,fips,0.0004,33
...,...,...,...
62,storydesc,0.9994,77364
6,basementsqft,0.9994,77364
16,finishedsquarefeet13,0.9995,77373
9,buildingclasstypeid,0.9998,77399


In [6]:
#see some stats on missing data
null_prop_df.describe()

,percent_null,count_null
count,68.000000,68.000000
mean,0.485854,37612.147059
std,0.448192,34696.103056
min,0.000000,0.000000
25%,0.000475,34.750000
50%,0.481200,37249.000000
75%,0.981600,75992.000000
max,0.999800,77399.000000


There is significant missing data in the rows.  A lot of rows will likely not be useful simply because of the amount of missing data.  However, the column of interest, `logerror`, contains no missing data.

<a href='#contents'>Back to contents</a>

### Prepare the Data <a name = 'prepare'></a>

#### Key Prepare Takeaways
- 28 columns and 51606 rows are retained representing $0.66$ of original data
- The majority of the loss is represented by droping non-single unit properties

#### Discussion

Consider the columns that have greater than $0.01$ of the rows having null values. Because there is significant data missing, all of these columns will be dropped.  Note, however, that some of these columns do have the majority of their data.  These columns could be used in a future analysis, see the <a href='#conclusion'>conclusion</a> for more discussion. 

This project is interested in single unit homes.  The column `propertylandusedesc` is filtered so that there is only the following values: `Single Family Residential`,`Mobile Hom`,`Manufactured, Modular, Prefabricated Home`,`Residential Genera`, and `Townhouse`.  Note that this is where the majority of the data is lost, as these rows represent 52495 of the original dataframe of 77414 rows (representing a loss of $0.32 = \frac{77414-52495}{77414}$.  There is a `unitcnt` column; however, it contains a significant number of nulls that are concentrated in Ventura and Orange counties.

The `fips` and `latitude`/`longitude` columns data are changed/corrected. The `fips` column is mapped to the strings `Los Angeles`, `Orange` and `Ventura`; it is then renamed `county`.  This aides in human readability. `latitude` and `longitude` are multiplied by $10^{-6}$, so that they represent the correct values for the Los Angeles metro area.

This is all done in the `wrangle_zillow.wrangle_data`, which takes a dataframe produced by `wrangle_zillow.get_zillow_data()` and calls the following functions:
- `filter_properties()` : filters the properties to only the above mentioned single unit properties.
- `handle_missing_values()` : Drops all rows and columns and columns that have greater and $0.01$ of data missing
- `clearing_fips()` : maps `fips` numbers to the correct county name and renames the column to `county`
- `latitude` and `longitude` are corrected by multiplying by $10^{-6}$ directly, not through a function
    
<a href='#contents'>Back to contents</a>

In [7]:
df = wrangle_zillow.wrangle_data(df) #wrangle the data
df.shape[0]/original_shape[0] #get the percentage of the original data retained

0.6666236081328959

$0.66$ of the data is retained after the above preparation.  The full info output on the dataframe is below:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51606 entries, 0 to 77413
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   logerror                      51606 non-null  float64
 1   transdate                     51606 non-null  object 
 2   id                            51606 non-null  int64  
 3   parcelid                      51606 non-null  int64  
 4   bathroomcnt                   51606 non-null  float64
 5   bedroomcnt                    51606 non-null  float64
 6   calculatedbathnbr             51606 non-null  float64
 7   calculatedfinishedsquarefeet  51606 non-null  float64
 8   finishedsquarefeet12          51606 non-null  float64
 9   fullbathcnt                   51606 non-null  float64
 10  latitude                      51606 non-null  float64
 11  longitude                     51606 non-null  float64
 12  lotsizesquarefeet             51606 non-null  float64
 13  p

<a href='#contents'>Back to contents</a>

## Exploration <a name='explore'></a>

### Key Explore Takeaways

### Splitting Data Into Subsets

The data is split into three subsets to prepare for exploration and modeling.  This is done with the `wrangle_zillow.split_data()` function which has the following parameters:
- `df` (DataFrame) : the dataframe to split
- `return_info` (bool) : returns a dataframe with the number of rows and columns of the returned dataframes.

In [9]:
# split the data into train, validate and test subset
train, validate, test, info_df = wrangle_zillow.split_data(df, return_info=True)
info_df.set_index('subset')

,rows,columns
subset,,
train,28898,28
validate,12386,28
test,10322,28


### Looking at the Target Variable: `logerror` <a name='target'></a>

#### Key Target Variable Takeaways